# The Fokas transform pairs

* Course: YSC4103 MCS Capstone
* Date created: 2018/10/07
* Name: Linfan XIAO
* Description: Implement the transform pair (2.15a), (2.15b) on page 10 of "Evolution PDEs and augmented eigenfunctions. Finite interval."

## Importing packages and modules

In [1]:
using SymPy
using QuadGK
# using HCubature
using ApproxFun
using Roots
using Gadfly
using PyPlot
pygui(true)
include("C:\\Users\\LinFan Xiao\\Academics\\College\\Capstone\\work_in_julia\\construct_adjoint.jl")

construct_validAdjoint (generic function with 1 method)

## Global variables

In [2]:
lambda = symbols("lambda")

lambda

## Helper functions

In [3]:
# Assign string as variable name
function assign(s::AbstractString, v::Any)
    s=Symbol(s)
    @eval (($s) = ($v))
end

# Function addition (f + g)(x) := f(x) + g(x)
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

# Function multiplication (f * g)(x) := f(x) * g(x)
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

mult_func (generic function with 1 method)

## Structs

## Main functions

In [4]:
# Get M+, M- in (2.13a), (2.13b) as functions of lambda (for fixed adjointU) as well as their symbolic expressions
function get_MPlusMinus(adjointU::VectorBoundaryForm; symbolic = false)
    # these are numeric matrices
    bStar, betaStar = adjointU.M, adjointU.N
    n = size(bStar)[1]
    alpha = e^(2pi*im/n)
    if symbolic
        # return MPlus and MMinus as symbolic expressions with lambda as free variable
        # global variable lambda
        MPlusMat = Array{SymPy.Sym}(n,n)
        for k = 1:n
            for j = 1:n
                sumPlus = 0
                for r = 0:(n-1)
                    summandPlus = (-im*alpha^(k-1)*lambda)^r * bStar[j,r+1]
                    summandPlus = expand(summandPlus)
                    sumPlus += summandPlus
                end
                sumPlus = expand(sumPlus) # for convenience when simplifying later
                MPlusMat[k,j] = sumPlus
            end
        end
        MPlusSym = MPlusMat
        MMinusMat = Array{SymPy.Sym}(n,n)
        for k = 1:n
            for j = 1:n
                sumMinus = 0
                for r = 0:(n-1)
                    summandMinus = (-im*alpha^(k-1)*lambda)^r * betaStar[j,r+1]
                    summandMinus = expand(summandMinus)
                    sumMinus += summandMinus
                end
                sumMinus = expand(sumMinus)
                MMinusMat[k,j] = sumMinus
            end
        end
        MMinusSym = MMinusMat
        return (MPlusSym, MMinusSym)
    else
        # if not symbolic, return MPlus and MMinus as functions of lambda
        function MPlus(lambda::Number)
            MPlusMat = Array{Number}(n,n)
            for k = 1:n
                for j = 1:n
                    sumPlus = 0
                    for r = 0:(n-1)
                        summandPlus = (-im*alpha^(k-1)*lambda)^r * bStar[j,r+1]
                        sumPlus += summandPlus
                    end
                    MPlusMat[k,j] = sumPlus
                end
            end
            return MPlusMat
        end
        function MMinus(lambda::Number)
            MMinusMat = Array{Number}(n,n)
            for k = 1:n
                for j = 1:n
                    sumMinus = 0
                    for r = 0:(n-1)
                        summandMinus = (-im*alpha^(k-1)*lambda)^r * betaStar[j,r+1]
                        sumMinus += summandMinus
                    end
                    MMinusMat[k,j] = sumMinus
                end
            end
            return MMinusMat
        end
    end
    return (MPlus, MMinus)
end

get_MPlusMinus (generic function with 1 method)

## Testing

In [5]:
t = symbols("t")
(a,b) = (0,1)
symPFunctions = [t+im t*im t]
pFunctions = [t->t+im t->t*im t->t]
symL = SymLinearDifferentialOperator(symPFunctions, (a,b), t)
L = LinearDifferentialOperator(pFunctions, (a,b), symL)
n = 2
MCandRe = rand(Uniform(1.0,10.0), n, n)
MCandIm = rand(Uniform(1.0,10.0), n, n)
MCand = MCandRe + MCandIm*im
NCandRe = rand(Uniform(1.0,10.0), n, n)
NCandIm = rand(Uniform(1.0,10.0), n, n)
NCand = NCandRe + NCandIm*im
U = VectorBoundaryForm(MCand, NCand)
pDerivMatrix = [t->t+im t->t; t->t*im t->im]
adjointU = construct_validAdjoint(L, U, pDerivMatrix)

VectorBoundaryForm(Complex[0.0-0.0im -5.5045e-17-1.0im; 0.0+1.0im -2.79979e-16+3.67206e-17im], Complex[-0.718651-0.379292im -0.371364-0.285078im; -1.6631+0.880004im -0.546284+0.567562im])

In [6]:
(MPlus, MMinus) = get_MPlusMinus(adjointU; symbolic = false)
MPlus

MPlus (generic function with 1 method)

In [7]:
(MPlus, MMinus) = get_MPlusMinus(adjointU; symbolic = true)
MPlus

2×2 Array{SymPy.Sym,2}:
 -1.0*lambda + 5.50450227910145e-17*I*lambda  …    3.6720621110913e-17*lambda + 2.79978897989928e-16*I*lambda + 1.0*I
   1.0*lambda - 1.7750970270575e-16*I*lambda     -3.67206211109131e-17*lambda - 2.79978897989928e-16*I*lambda + 1.0*I

In [8]:
simplify(SymPy.det(MPlus))

lambda*(6.16297582203915e-33*lambda - 2.32554725496764e-16 - 2.0*I)